In [1]:
from flask import Flask
import eventlet.wsgi
import eventlet
import socketio
import base64
from PIL import Image
from io import BytesIO 
import numpy as np
import matplotlib.pyplot as plt
from keras.models import load_model

Using TensorFlow backend.


In [2]:
sio = socketio.Server()
app = Flask(__name__)

def send_control(steering_angle, throttle):
    sio.emit("steer", data={'steering_angle': str(steering_angle),
                            'throttle': str(throttle) }, skip_sid=True)


In [3]:
model = load_model('model.h5')
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 58, 78, 64)        1792      
_________________________________________________________________
activation_1 (Activation)    (None, 58, 78, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 76, 32)        18464     
_________________________________________________________________
activation_2 (Activation)    (None, 56, 76, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 28, 38, 32)        0         
_______________________________________

In [ ]:
def process_image(img):
    #160,320,3 ==> 60,80,3
    return img[10:130:2, ::4, :]




@sio.on('telemetry')
def telemetry(sid, data):
    if data:
        speed = float(data["speed"])
        image_str = data["image"]
         
        decoded = base64.b64decode(image_str)
        image = Image.open(BytesIO(decoded))
        image_array = np.asarray(image)
        
        img = process_image(image_array)
        img_batch = np.expand_dims(img, axis=0)
        
        
        steering_angle = float(model.predict(img_batch))
        
        

        throttle = 0.17
    
        if speed < 15:
            throttle = 0.7
        if speed > 17:
            throttle = -0.2

        send_control(steering_angle, throttle)
    else:
        sio.emit('manual', data={}, skip_sid=True)

app = socketio.Middleware(sio, app)
eventlet.wsgi.server(eventlet.listen(('', 4567)), app)

(12280) wsgi starting up on http://0.0.0.0:4567
(12280) accepted ('127.0.0.1', 54931)
(12280) accepted ('127.0.0.1', 54932)
